In [1]:
import tweepy
import re
import csv
from tweepy import OAuthHandler
from textblob import TextBlob

In [4]:
class Twitter():
    ##Twitter class for sentiment analysis##

    def __init__(self):
        ##Authorising Twitter API client##
    
        consumer_key = "XXX"    # API key
        consumer_secret = "XXX"   # API secret key  
        access_token = "XXX"
        access_token_secret = "XXX"

        try:
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            self.auth.set_access_token(access_token, access_token_secret)
            self.api = tweepy.API(self.auth)
        except:
            print("Authentication Failed")

    def preprocess_tweet(self, tweet):
        ##Preprocessing a tweet##

        link_ex = "https?:\/\/[A-Za-z0-9./]+"
        handle_ex = "@[A-Za-z0-9_]+"
        hashtag_ex = "#[A-Za-z0-9_]+"
        spcl_ex = "[^0-9A-Za-z ]"

        tweet = re.sub(link_ex, " ", tweet)         #removing links
        tweet = re.sub(handle_ex, " ", tweet)       #removing twitter handles
        tweet = re.sub(hashtag_ex, " ", tweet)      #removing hashtags
        tweet = re.sub(spcl_ex, " ", tweet)         #removing special characters
        
        tweet = ''.join([i if ord(i) < 128 else ' ' for i in tweet])    #removing non-ascii characters

        return tweet

    def clean_tweet_text(self, tweet_text):
        ##Cleaning tweet to store in the CSV file##
        
        spcl_ex = "[^0-9A-Za-z .@#!?]"
        tweet_text = re.sub(spcl_ex, " ", tweet_text)       #removing special characters
        tweet_text = ''.join([i if ord(i) < 128 else ' ' for i in tweet_text])  #removing non-ascii characters

        return tweet_text

    def get_tweet_sentiment(self, tweet):
        ##Getting sentiment of a tweet##

        tweet_stats = TextBlob(self.preprocess_tweet(tweet))
        if tweet_stats.sentiment.polarity < 0:
            return "negative"
        elif tweet_stats.sentiment.polarity == 0:
            return "neutral"
        else:
            return "positive"

    def get_tweets(self, topic, tweet_count):
        ##GET request to retrieve english tweets on a particular topic##

        tweets = list()
        try:
            matched_tweets = self.api.search(q = topic, lang='en', show_user = False, count = tweet_count)
            for tweet in matched_tweets:
                parsed_tweet = {}
                parsed_tweet['text'] = self.clean_tweet_text(tweet.text)
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)

                #Checking if the tweet is already present in our list because of it being retweeted
                if tweet.retweet_count > 0:
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
            return tweets
        except tweepy.TweepError as e:
            print("Error: " +str(e))

In [7]:
def main():
    my_api = Twitter()
    topic = "avengers"
    tweets = my_api.get_tweets(topic, tweet_count = 100)

    for tweet in tweets:
      print(tweet['text'] + " --------->> " + tweet['sentiment'] + "\n\n")

    pos_tweets = [t for t in tweets if t['sentiment'] == 'positive']
    neg_tweets = [t for t in tweets if t['sentiment'] == 'negative']
    neut_tweets_num = len(tweets)-len(pos_tweets)-len(neg_tweets)

    pos_percentage = float(100*len(pos_tweets)/len(tweets))
    neg_percentage = float(100*len(neg_tweets)/len(tweets))
    neut_percentage = float(100*neut_tweets_num/len(tweets))

    print("\n\n---Aggregate Analysis for the topic: " + topic + " ---\n")
    print("1. Percentage of POSITIVE tweets -> " + str(pos_percentage) + "%")
    print("2. Percentage of NEGATIVE tweets -> " + str(neg_percentage) + "%")
    print("3. Percentage of NEUTRAL tweets -> " + str(neut_percentage) + "%\n\n")
    
#   keys = tweets[0].keys()
#   with open("Sentiment_Analysis_TOPIC.csv", "wb") as f:
#       w = csv.DictWriter(f,keys)
#       w.writeheader()
#       w.writerows(tweets)
#   print("\n\n\nCSV file named \"Sentiment_Analysis_TOPIC.csv\" created for individual analysis!\n\n")

In [8]:
if __name__ == "__main__": 
    main()

RT @missingabhijeet  Get ready to witness World Class Action. Because the action director of Hollywood movies like Mission Impossible  Top  --------->> positive


ay to they got Portugal. The Man  Paramore  Blink 182  Ashnikko  Avril Lavigne  Beach Boys  MCR  Queen  Cyndi Laupe  https   t.co Y8cBCLKf2W --------->> neutral


RT @fxala2  BACK TO THE ORIGINS # 27  Colored   amp  final  #Xmen #Avengers #Marvel #ferranxalabarder https   t.co ItqdizLljQ --------->> neutral


Get ready to witness World Class Action. Because the action director of Hollywood movies like Mission Impossible  T  https   t.co 8V8QBn7FKo --------->> negative


RT @IronNinja2000  Avengers budget is nonexistent at this point... my god... https   t.co i5mafrQepc --------->> neutral


RT @kanemoyoshi  the way that treasure s video background for mama performance looks like it got from avengers. i know that it might was hy  --------->> neutral


@Harshav74123148 No bro. Avengers Endgame grossed  62 678 476 in India which